# Machine Learning Programming - PROG8245
## Lab 2 - Data Collection and Pre-processing

This notebook implements a 12-step data engineering pipeline for an e-commerce dataset. It loads, cleans, transforms, and analyzes synthetic transaction data, then merges it with a secondary metadata source to create a data dictionary. All steps include explanations and follow the assignment requirements.

## STEP 1: Hello, Data!

Load the synthetic e-commerce CSV file and display the first 3 rows to verify the data.

In [2]:
import pandas as pd
import random
import datetime
from faker import Faker

# Initialize Faker for Canadian data
fake = Faker("en_CA")

# Define lists for synthetic data
Products_list = [
    'Blender', 'Refrigerator', 'Desk Chair', 'Vacuum Cleaner', 'Microwave',
    'VacuumCleaner', 'Sofa', 'Coffee Maker', 'Air Conditioner', 'Bookshelf',
    'Dining Table'
]
Coupons_list = ['SAVE10', 'WELCOME15', 'FREESHIP', 'HOLIDAY20', 'SPRING5', 'save-10', None]
City_list = ['Toronto', 'Vancouver', 'Montreal', 'Calgary', 'Ottawa', 'Edmonton',
             'QuebecCity', 'Winnipeg', 'Hamilton', 'Kitchener']

# Define date range
start = datetime.date(2015, 1, 1)
end = datetime.date(2024, 6, 1)

# Generate 500 rows of synthetic data
rows = []
for i in range(500):
    Product = random.choice(Products_list)
    Price = round(random.uniform(500, 2000), 2)
    city = random.choice(City_list)
    coupon = random.choice(Coupons_list)
    
    row = {
        "Date": fake.date_between(start_date=start, end_date=end),
        "CustomerID": fake.random_int(min=1, max=500),
        "Product": Product,
        "Price": Price,
        "Quantity": random.randint(1, 7),
        "Coupon": coupon,
        "Shipping_city": city
    }
    rows.append(row)

# Create DataFrame and save to CSV
data = pd.DataFrame(rows)
data.to_csv('../Data/Ecommerce2025.csv', index=False)

# Display first 3 rows
print(data.head(3))

         Date  CustomerID         Product    Price  Quantity     Coupon  \
0  2019-11-07         473   VacuumCleaner  1598.86         1  WELCOME15   
1  2018-10-08          20            Sofa  1407.86         3    save-10   
2  2018-11-30         406  Vacuum Cleaner   622.09         7  HOLIDAY20   

  Shipping_city  
0      Hamilton  
1     Vancouver  
2     Kitchener  


## STEP 2: Pick the Right Container	
I will use dictionaries beacuse my dataset has attributes and values , and dictionaries store data as key value pair. this thing will make it easy to access each field using its lable and also easy to add/remove fields later. set will be useful when we are using to store the unique items, one example such as distinct list of cities. while namedtuple is better when the data fields are fixed and unchanging. By analysing this i would prefer a dictionary

## STEP 3: Implement Functions and Data Structure


In [ ]:
from dataclasses import dataclass

@dataclass
class Transaction:
    Date: pd.Timestamp
    CustomerID: int
    Product: str
    Price: float
    Quantity: int
    Coupon: str
    Shipping_city: str
    
    def clean(self):
        if self.Product == 'VacuumCleaner':
            self.Product = 'Vacuum Cleaner'
   
        if self.Coupon == 'save-10':
            self.Coupon = 'SAVE10'
       
        if self.Coupon is None or self.Coupon == '':
            self.Coupon = 'NONE'
        
        if self.Shipping_city == 'QuebecCity':
            self.Shipping_city = 'Quebec City'
    
    def total(self):
        return self.Price * self.Quantity

## STEP 4: Bulk Loaded

Load the CSV into a list of dictionaries for processing.

In [ ]:
# Load CSV
data = pd.read_csv('../Data/Ecommerce2025.csv')
ecommerce_data = data.to_dict(orient='records')

# Apply cleaning 
for record in ecommerce_data:
    trans = Transaction(
        Date=pd.to_datetime(record['Date']),
        CustomerID=record['CustomerID'],
        Product=record['Product'],
        Price=record['Price'],
        Quantity=record['Quantity'],
        Coupon=record['Coupon'],
        Shipping_city=record['Shipping_city']
    )
    trans.clean()
    record['Product'] = trans.Product
    record['Coupon'] = trans.Coupon
    record['Shipping_city'] = trans.Shipping_city

# Print first 3 records to verify
print(ecommerce_data[:3])

[{'Date': '2019-11-07', 'CustomerID': 473, 'Product': 'Vacuum Cleaner', 'Price': 1598.86, 'Quantity': 1, 'Coupon': 'WELCOME15', 'Shipping_city': 'Hamilton'}, {'Date': '2018-10-08', 'CustomerID': 20, 'Product': 'Sofa', 'Price': 1407.86, 'Quantity': 3, 'Coupon': 'SAVE10', 'Shipping_city': 'Vancouver'}, {'Date': '2018-11-30', 'CustomerID': 406, 'Product': 'Vacuum Cleaner', 'Price': 622.09, 'Quantity': 7, 'Coupon': 'HOLIDAY20', 'Shipping_city': 'Kitchener'}]


## STEP 5: Quick Profiling

Calculate min, max, and mean for Price and Quantity, and count unique products and cities.

In [ ]:
data = pd.DataFrame(ecommerce_data)

min_price = data['Price'].min()
max_price = data['Price'].max()
mean_price = data['Price'].mean()
print(f"Price - Min: {min_price:.2f} Max: {max_price:.2f} Mean: {mean_price:.2f}")
print("-----")

min_quantity = data['Quantity'].min()
max_quantity = data['Quantity'].max()
mean_quantity = data['Quantity'].mean()
print(f"Quantity - Min: {min_quantity} Max: {max_quantity} Mean: {mean_quantity:.2f}")
print("-----")

unique_products_count = data['Product'].nunique()
unique_cities_count = data['Shipping_city'].nunique()
print(f"Count of Unique Products: {unique_products_count}")
print(f"Count of Unique Cities: {unique_cities_count}")
print("-----")

unique_cities = data['Shipping_city'].unique()
print("Unique Cities:", unique_cities)
print("-----")

unique_products = data['Product'].unique()
print("Unique Products:", unique_products)
print("-----")

Price - Min: 502.74 Max: 1999.87 Mean: 1253.16
-----
Quantity - Min: 1 Max: 7 Mean: 3.92
-----
Count of Unique Products: 10
Count of Unique Cities: 10
-----
Unique Cities: ['Hamilton' 'Vancouver' 'Kitchener' 'Montreal' 'Winnipeg' 'Toronto'
 'Quebec City' 'Edmonton' 'Ottawa' 'Calgary']
-----
Unique Products: ['Vacuum Cleaner' 'Sofa' 'Coffee Maker' 'Dining Table' 'Desk Chair'
 'Air Conditioner' 'Microwave' 'Blender' 'Bookshelf' 'Refrigerator']
-----


## STEP 6: Spot the Grime

Three dirty data issues identified:
1. **Inconsistent Product Names**: 'VacuumCleaner' and 'Vacuum Cleaner' refer to the same product.
2. **Inconsistent Coupon Codes**: 'save-10' and 'SAVE10' are the same but formatted differently.
3. **Missing/None Coupons**: Some records have `None` or empty strings for coupons, which should be standardized to 'NONE'.
4. **Inconsistent City Names**: 'QuebecCity' should be 'Quebec City' for consistency.

## STEP 7: Cleaning Rules


In [ ]:
raw_data = pd.read_csv('../Data/Ecommerce2025.csv')

print("Before Cleaning:")
print(f"Vacuum Cleaner count: {len(raw_data[raw_data['Product'] == 'Vacuum Cleaner'])}")
print(f"VacuumCleaner count: {len(raw_data[raw_data['Product'] == 'VacuumCleaner'])}")
print(f"save-10 count: {len(raw_data[raw_data['Coupon'] == 'save-10'])}")
print(f"SAVE10 count: {len(raw_data[raw_data['Coupon'] == 'SAVE10'])}")
print(f"None/empty coupon count: {len(raw_data[raw_data['Coupon'].isna() | (raw_data['Coupon'] == '')])}")
print(f"QuebecCity count: {len(raw_data[raw_data['Shipping_city'] == 'QuebecCity'])}")

# After cleaning counts (using cleaned data)
print("\nAfter Cleaning:")
print(f"Vacuum Cleaner count: {len(data[data['Product'] == 'Vacuum Cleaner'])}")
print(f"VacuumCleaner count: {len(data[data['Product'] == 'VacuumCleaner'])}")
print(f"save-10 count: {len(data[data['Coupon'] == 'save-10'])}")
print(f"SAVE10 count: {len(data[data['Coupon'] == 'SAVE10'])}")
print(f"NONE coupon count: {len(data[data['Coupon'] == 'NONE'])}")
print(f"Quebec City count: {len(data[data['Shipping_city'] == 'Quebec City'])}")

Before Cleaning:
Vacuum Cleaner count: 36
VacuumCleaner count: 46
save-10 count: 79
SAVE10 count: 67
None/empty coupon count: 92
QuebecCity count: 50

After Cleaning:
Vacuum Cleaner count: 82
VacuumCleaner count: 0
save-10 count: 0
SAVE10 count: 146
NONE coupon count: 0
Quebec City count: 50


## STEP 8: Transformations

In [ ]:
# Define coupon discount mapping
coupon_discounts = {
    'SAVE10': 10,
    'WELCOME15': 15,
    'FREESHIP': 0,  
    'HOLIDAY20': 20,
    'SPRING5': 5,
    'NONE': 0
}
data['Discount_Percent'] = data['Coupon'].map(coupon_discounts)

# Verify
print(data['Discount_Percent'].value_counts().reset_index(name='Count'))

   Discount_Percent  Count
0              0.0     76
1              5.0     74
2             10.0    140
3             15.0     71
4             20.0     71


## STEP 9: Feature Engineering

Adding a "Days_Since_Purchase" column, calculating days from the transaction date to today (2025-09-30).

In [ ]:
data['Date'] = pd.to_datetime(data['Date'])

current_date = pd.to_datetime('2025-09-30')
data['Days_Since_Purchase'] = (current_date - data['Date']).dt.days
# Verify new column
print(data.head(3))

        Date  CustomerID         Product    Price  Quantity     Coupon  \
0 2019-11-07         473  Vacuum Cleaner  1598.86         1  WELCOME15   
1 2018-10-08          20            Sofa  1407.86         3     SAVE10   
2 2018-11-30         406  Vacuum Cleaner   622.09         7  HOLIDAY20   

  Shipping_city  Days_Since_Purchase  
0      Hamilton                 2154  
1     Vancouver                 2549  
2     Kitchener                 2496  


## STEP 10: Mini-Aggregation

Calculate total revenue (Price * Quantity) per shipping city using pandas groupby.

In [10]:
# Calculate Total_Revenue
data['Total_Revenue'] = data['Price'] * data['Quantity']

# Aggregate by shipping city
revenue_by_city = data.groupby('Shipping_city')['Total_Revenue'].sum()
print(revenue_by_city)

Shipping_city
Calgary        239538.36
Edmonton       266903.26
Hamilton       166577.53
Kitchener      226744.13
Montreal       259269.19
Ottawa         227216.47
Quebec City    240882.69
Toronto        263547.24
Vancouver      231393.03
Winnipeg       313963.21
Name: Total_Revenue, dtype: float64


## STEP 11: Serialization Checkpoint

Save the cleaned and transformed data to JSON.

In [11]:
# Save to JSON
data.to_json('../Data/Ecommerce2025_cleaned.json', orient='records', date_format='iso')

## STEP 12: Soft Interview Reflection

Functions helped to make the pipeline more efficient by splitting activities like cleaning and totaling into reusable functions within the `Transaction` class. The `clean` function cleaned up inconsistencies (e.g., product names) efficiently for all records. The `total` function facilitated easy calculation of revenues. Through the use of functions, duplicated code was eliminated, debugging was simplified, and the notebook stayed organized. This modular design also facilitated easy addition of new features, like discount parsing, without the need to rewrite code.

## Data Dictionary

Merged from primary CSV (Ecommerce2025.csv) and secondary metadata (coupon descriptions from a hypothetical source: https://example.com/coupon_metadata),(table created with the help of AI)

| Field                | Type       | Description                                                                 | Source                          |
|----------------------|------------|-----------------------------------------------------------------------------|---------------------------------|
| Date                 | datetime   | Date of transaction (YYYY-MM-DD)                                            | Primary CSV (synthetic)         |
| CustomerID           | integer    | Unique customer identifier (1–500)                                          | Primary CSV (synthetic)         |
| Product              | string     | Product purchased (e.g., Blender, Sofa)                                     | Primary CSV (synthetic)         |
| Price                | float      | Price per unit (500–2000 CAD)                                              | Primary CSV (synthetic)         |
| Quantity             | integer    | Number of units purchased (1–7)                                             | Primary CSV (synthetic)         |
| Coupon               | string     | Coupon code applied (e.g., SAVE10, NONE)                                    | Primary CSV (synthetic)         |
| Shipping_city        | string     | City of delivery (e.g., Toronto, Quebec City)                               | Primary CSV (synthetic)         |
| Discount_Percent     | float      | Discount percentage from coupon (0–20%)                                     | Derived (Step 8, coupon mapping)|
| Days_Since_Purchase  | integer    | Days from transaction to 2025-09-30                                        | Derived (Step 9, date diff)     |
| Total_Revenue        | float      | Total revenue (Price * Quantity)                                           | Derived (Step 10, calculation)  |

**Notes**: 
- Primary fields (Date, CustomerID, etc.) were generated synthetically using Faker.
- "Discount_Percent" was created by mapping coupon codes to percentages based on a hypothetical metadata source.
- "Days_Since_Purch"ase was calculated as the difference between transaction date and 2025-09-30.
- "Total_Revenue" was computed as Price * Quantity.